# Developing a compact Blood Bowl game notation using FUMBBL replay data

the botbowl ppl are discussing writing a parser for the json that would result in log with chess like notation ie P1 - B - E4 to C5 - (player one chooses blitz, moves from position e4 to c5 etc.

This will be the focus of this notebook.

# Replay files: What has already been done

Christian Huber (aka Candlejack) has two repo's open that are of great interest.

First is https://github.com/SanityResort/htmlreplay

This connect to the FFB server and requests a replay file that it reads in and converts to JSON.
The goal seems to be to be able to visualize a replay in the browser via HTML.

Then there is https://github.com/SanityResort/FFBStats
This is code that processes replay files and extracts information from them. Exactly what we want as well!
It was written in 2016 and integrated into the site in 2017 by Christer. Then went away and came back in 2022.
It produces a match statistics file as JSON, that forms the basis of a nice visualization on the match result page on FUMBBL.
The match statistics are available through the API.

https://fumbbl.com/p/match?op=stats&id=3916966

# FUMBBL Replay datafiles: opening up the black box

Replay data is quite verbose and there’s lots of it. The FFB client communicates with the Server using Java Web Sockets.
The raw data packages send over the line are web sockets. 
A replay file would be the json command stream. It’s a complex format, as it’s more or less just logging the data packets between the client and server. 

For example, after unzipping, opening as JSON in VScode and doing autoformatting, we end up with a 266K lines of client server "command" stream.
Each turn is about 10K lines, with roster info at the end.

The high level file format is as follows:

```
{
    "gameStatus": "uploaded",
    "stepStack": {
        "steps": []
    },
    "gameLog": {}, # contains the command stream
    "game": {}, # contains the full roster and position information
    "playerIds": [],
    "swarmingPlayerActual": 0,
    "passState": {},
    "prayerState": {},
    "activeEffects": {}
}
```

I noticed that the replays are really nicely self contained, they contain full copies of the rosters and (if i remember correctly) even ruleset.
Everything in the ruleset that has a bearing on the client
Basically the "client options" tab of the ruleset

A full history of all the events during the game is stored under `gameLog`.

The various phases of the match are clearly distinguished in the command streams.
`turnDataSetTurnNr` , `turnDataSetFirstTurnAfterKickoff`, `gameSetTurnMode`, etc.


## netCommands that communicate changes

The basic unit is the command, that is indexed by `commandNr`. 
A match consists of several thousand commands.
A typical command has the following **FIXED** structure:

```
{
    "netCommandId": "serverModelSync",
    "commandNr": 243,
    "modelChangeList": {
        "modelChangeArray": []
    },
    "reportList": {
        "reports": []
    },
    "sound": null,
    "gameTime": 805789,
    "turnTime": 179506
}
```

`modelChange` changes the game state.
`reportList` directs output to the client's reporting panel.



## The Field model including the coordinate system

FFB uses a field model that is very straightforward. The 15 x 26 game board is indexed using (X,Y) coordinates, with the top left square being (0,0). 
and the lower right square being (25, 14). 

Players can be either:
* On the pitch
* In the reserve box
* IN the KO box
* In the Badly hurt box
* In the Seriously injured box
* In the RIP box
* In the Ban box
* In The miss next game box

On the pitch the X,Y coordinates are used, the other locations are indexed using -1,-2,-3 etc. 

```
export default class Coordinate {
    x: number;
    y: number;

    public static FIELD_WIDTH = 26;
    public static FIELD_HEIGHT = 15;
    public static RSV_HOME_X = -1;
    public static KO_HOME_X = -2;
    public static BH_HOME_X = -3;
    public static SI_HOME_X = -4;
    public static RIP_HOME_X = -5;
    public static BAN_HOME_X = -6;
    public static MNG_HOME_X = -7;
    public static RSV_AWAY_X = 30;
    public static KO_AWAY_X = 31;
    public static BH_AWAY_X = 32;
    public static SI_AWAY_X = 33;
    public static RIP_AWAY_X = 34;
    public static BAN_AWAY_X = 35;
    public static MNG_AWAY_X = 36;
    
}
```

FFB uses `fieldModelSetPlayerCoordinate` to position players on the field or on the dug out. Players are identified using the FUMBBL player ids. At the end of the replay, all the player information is stored, including extra skills above those that come with the positional, as well as the full rosters (including all possible star players).


## The list of Player states

PlayerState contains Player State encoded as x-bit integer, with State modifiers (i.e. a PRONE that is also ROOTED) as higher bits that can be set.

```
export default class PlayerState {
    public static UNKNOWN = 0;
    public static STANDING = 1;
    public static MOVING = 2;
    public static PRONE = 3;
    public static STUNNED = 4;
    public static KNOCKED_OUT = 5;
    public static BADLY_HURT = 6;
    public static SERIOUS_INJURY = 7;
    public static RIP = 8;
    public static RESERVE = 9;
    public static MISSING = 10;
    public static FALLING = 11;
    public static BLOCKED = 12;
    public static BANNED = 13;
    public static EXHAUSTED = 14;
    public static BEING_DRAGGED = 15;
    public static PICKED_UP = 16;
    public static HIT_BY_FIREBALL = 17;
    public static HIT_BY_LIGHTNING = 18;
    public static HIT_BY_BOMB = 19;
    public static BIT_ACTIVE = 256;
    public static BIT_CONFUSED = 512;
    public static BIT_ROOTED = 1024;
    public static BIT_HYPNOTIZED = 2048;
    public static BIT_BLOODLUST = 4096;
    public static BIT_USED_PRO = 8192
}
```

# A flat table format extracted from replays

This could be the basis of our flat file format.

We write a for loop that cycles through the commands, and fills out a pandas dataframe.
We use what we learned with the API data.

Columns in our initial data format:

* commandNr
* modelChangeId
* modelChangeKey
* playerId
* playerState
* playerXcoordinate
* playerYcoordinate
* gameTime
* turnTime

if `modelChangeId` equals `fieldModelSetPlayerState` we record the `modelChangeValue` under `playerState`, and if it equals `fieldModelSetPlayerCoordinate` we record the `modelChangeValue` vector under `playerXcoordinate` and `playerYcoordinate`. In both cases the `PlayerId` can be found under `modelChangeKey`.


# Develop as Python package

The development environment consists of VScode editor, with a Python virtual environment `requests_env` attached. Here the code is installed as Python package `fumbbl_bbgn` (from FUMBBL replay to BBGN format). Whenever the code is changed, I run `python3 -m pip install -e .` from a VScode python terminal with the Venv activated, to install the updated package.
I use a Jupyter notebook to import the package and run the `main_program` function.

# Load packages

# Parse replay, write to Excel

Using the API we fetch the replay file for this match:
https://fumbbl.com/p/match?id=4407782

Goal: taking a two step approach here, first generate a complete but not redundant game log with all the glorious details that a BB match can have, and when we have that, we can start, line by line, to map that on a compact annotation that would be feasible to write down with pen and paper during a table top game. 

main stuff:
-make parser for blockroll during blitz, 
-pickupproll, leaproll, handover, apothecaryRoll, kick-off table
-use Player State codes 3-7 for injuries

small stuff:
-add activeTeam column
-leave out activate/deactivate (ie coach changed his mind)


-cid 839/840: 2 times same block roll   -> first choice for reroll, then opponent chooses the result. 
-cid845 apo KO becomes stun (playerState 4)

bugs:
-KO/CAS should be ignored when converting to letter-number positions
-KO now is position a32 cid917

checked up until cid978, start turn 5.

testing:
-add test cases 
-Validate on new match https://fumbbl.com/FUMBBL.php?page=match&id=4444067)
-set up venv with requirements.txt (use docker?)


In [ ]:
import fumbbl_replays as fb
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.options.mode.chained_assignment = None

#dff = fb.fumbbl2bbgn(replay_id = 1559380)
dff = fb.fumbbl2bbgn(replay_id = 1712868)


# References

* Planning in the midst of chaos: how a stochastic Blood Bowl model can help to identify key planning features

* STARDATA: A StarCraft AI Research Dataset